Import of Packages

In [1]:
import scipy.stats as stats
import heapq
from collections import defaultdict
#from Code.import_data import import_data
from datetime import datetime, timedelta
import pandas as pd

Import of Data

In [400]:

def import_data():
    # @diana and denis, change to your directory
    gtfs_dir = '/Users/paulinaheine/Codes/BackupTravelPlanning/GTFS_OP_2024_obb-2/'
    #gtfs_dir = '/Users/paulinaheine/Codes/BackupTravelPlanning/GTFS_OP_2025_obb/'

    # Load data
    agency_df = pd.read_csv(gtfs_dir + 'agency.txt')
    stops_df = pd.read_csv(gtfs_dir + 'stops.txt')
    routes_df = pd.read_csv(gtfs_dir + 'routes.txt')
    trips_df = pd.read_csv(gtfs_dir + 'trips.txt')
    stop_times_df = pd.read_csv(gtfs_dir + 'stop_times.txt')
    calendar_df = pd.read_csv(gtfs_dir + 'calendar.txt')
    calendar_dates_df = pd.read_csv(gtfs_dir + 'calendar_dates.txt')
    shapes_df = pd.read_csv(gtfs_dir + 'shapes.txt')
    
    # Return all DataFrames
    return agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df, calendar_dates_df, shapes_df

# Import and display data


Helping Functions

In [384]:
def time_to_minutes(time_str):
    hours, minutes, seconds = map(int, time_str.split(":"))
    return hours * 60 + minutes + seconds / 60

def minutes_to_time(minutes):
    hours = int(minutes // 60)
    minutes = int(minutes % 60)
    return f"{hours:02d}:{minutes:02d}"

def get_weekday(date):
    weekdays = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    return weekdays[date.weekday()]

def get_available_service_ids(start_date, calendar, calendar_dates):
    """
    Gibt eine Liste der verfügbaren Service-IDs für das angegebene Datum zurück.
    """
    # Konvertiere start_date zu einem Datum im Format YYYYMMDD
    start_date_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    start_date_str = start_date_datetime.strftime("%Y%m%d")
    weekday = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"][start_date_datetime.weekday()]

    available_service_ids = []
    
    
        # 🔹 Schritt 2: Reguläre Services aus `calendar` prüfen
    for _, service in calendar.iterrows():
        service_id = service["service_id"]

        # Prüfen, ob das Datum im gültigen Zeitraum liegt
        if int(service["start_date"]) <= int(start_date_str) <= int(service["end_date"]):
            # Prüfen, ob der Service an diesem Wochentag aktiv ist
            if service[weekday] == 1:
                if service_id not in available_service_ids:
                    available_service_ids.append(service_id)
                   

    # 🔹 Schritt 1: Sonderregelungen aus `calendar_dates` berücksichtigen
    exceptions = calendar_dates[calendar_dates["date"] == int(start_date_str)]
    
    for _, exception in exceptions.iterrows():
        service_id = exception["service_id"]
        if exception["exception_type"] == 2:  # Service wird als Ausnahme hinzugefügt
            if service_id not in available_service_ids:
                available_service_ids.append(service_id)
        elif exception["exception_type"] == 1:  # Service wird als Ausnahme entfernt
            if service_id in available_service_ids:
                available_service_ids.remove(service_id)
    return available_service_ids 


    

def prepare_calendar_dates(calendar_dates):
    grouped = calendar_dates.groupby("service_id")
    calendar_dates_dict = {}
    for service_id, group in grouped:
        exceptions = group.to_dict(orient="records")
        calendar_dates_dict[service_id] = exceptions
    return calendar_dates_dict

In [401]:
agency, stops, routes, trips, stop_times, calendar, calendar_dates, shapes = import_data()

Create Graph

In [365]:
def create_graph_with_schedule(stop_times, stops, trips, calendar, calendar_dates, start_time_obj, end_time_obj):
    """
    Erstellt einen Graphen mit Knoten basierend auf verfügbaren Fahrten und Zeitfenstern.
    """

    from collections import defaultdict

    graph = defaultdict(list)
    stop_id_to_name = stops.set_index("stop_id")["stop_name"].to_dict()

    # 🔹 Hole alle gültigen Service-IDs für den Tag
    available_services = get_available_service_ids(start_time_obj.strftime("%Y-%m-%d"), calendar, calendar_dates)
    print(len(available_services))

    # 🔹 Filtere `stop_times` basierend auf der Zeitspanne
    stop_times = stop_times.sort_values(by=["trip_id", "stop_sequence"])
    stop_times["arrival_minutes"] = stop_times["arrival_time"].apply(time_to_minutes)
    stop_times["departure_minutes"] = stop_times["departure_time"].apply(time_to_minutes)
    trip_id_to_service = trips.set_index("trip_id")["service_id"].to_dict()
    stop_times["service_id"] = stop_times["trip_id"].map(trip_id_to_service)

    start_minutes = time_to_minutes(start_time_obj.strftime("%H:%M:%S"))
    end_minutes = time_to_minutes(end_time_obj.strftime("%H:%M:%S"))

    stop_times_filtered = stop_times[
        (stop_times["arrival_minutes"] >= start_minutes) &
        (stop_times["departure_minutes"] <= end_minutes) &
        (stop_times["service_id"].isin(available_services))
    ]

    print(f"Rows after time window and service filter: {len(stop_times_filtered)}")

    # 🔹 Graphen erstellen
    trip_id_to_route = trips.set_index("trip_id")["route_id"].to_dict()
    grouped = stop_times_filtered.groupby("trip_id")

    for trip_id, group in grouped:
        stops_list = group["stop_id"].tolist()
        departures = group["departure_time"].apply(time_to_minutes).tolist()
        arrivals = group["arrival_time"].apply(time_to_minutes).tolist()

        for start, end, dep, arr in zip(stops_list[:-1], stops_list[1:], departures[:-1], arrivals[1:]):
            travel_time = arr - dep
            if travel_time > 0:
                graph[stop_id_to_name[start]].append(
                    (stop_id_to_name[end], dep, arr, trip_id_to_route.get(trip_id))
                )

    return graph


Reliability function

In [247]:
def compute_transfer_probability_with_departure_delay(transfer_time):
    return min(0.90, stats.gamma.cdf(transfer_time, a=2, scale=3))


In [115]:
def compute_total_reliability(reliability_fast, backup_routes):
    """
    Berechnet die Gesamtzuverlässigkeit der gesamten Route gemäß Wahrscheinlichkeitslogik.

    Parameters:
        primary_reliability: Zuverlässigkeit der primären Route (rel_i).
        backup_routes: Liste der Backup-Routen mit deren Zuverlässigkeit.

    Returns:
        Die gesamte Zuverlässigkeit der Route (rel_{\bar{i}}).
    """
    total_reliability = reliability_fast
    remaining_probability = 1 - reliability_fast  # Wahrscheinlichkeit, dass die Primärroute fehlschlägt

    for backup in backup_routes:
        backup_reliability = backup[2]
        total_reliability += backup_reliability * remaining_probability
        remaining_probability *= (1 - backup_reliability)

    return total_reliability
#zwischen 0 und 1


# A* 

In [591]:
import heapq
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius der Erde in Kilometern
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # Entfernung in Kilometern

def heuristic(current_stop, end_name, stops_df):
    current_stop_row = stops_df[stops_df['stop_name'] == current_stop].iloc[0]
    end_stop_row = stops_df[stops_df['stop_name'] == end_name].iloc[0]

    current_lat = current_stop_row['stop_lat']
    current_lon = current_stop_row['stop_lon']
    end_lat = end_stop_row['stop_lat']
    end_lon = end_stop_row['stop_lon']

    distance_km = haversine(current_lat, current_lon, end_lat, end_lon)

    average_speed = 60  # km/h
    estimated_time_minutes = (distance_km / average_speed) * 60
    return estimated_time_minutes

#heuristic("Wien Handelskai", end_stop_name,  stops)

def a_star(graph, start_name, end_name, start_time_minutes, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=2, stops_df=None):
    pq = [(start_time_minutes, start_name, [], 1.0, None, 0)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie, Heuristik)
    visited = set()
    best_result = None  # Variable, um das beste Ergebnis zu speichern

    while pq:
        current_time, current_stop, path, reliability, last_route, _ = heapq.heappop(pq)

        if (current_stop, current_time) in visited:
            continue
        visited.add((current_stop, current_time))

        path = path + [(current_stop, current_time)]

        # Überprüfe, ob das Ziel erreicht wurde
        if current_stop == end_name:
            # Hier speichern wir das erste Ergebnis, wenn das Ziel erreicht wurde
            if best_result is None or new_current_time < best_result[0]:  # Wenn das Ziel früher erreicht wird
                best_result = (current_time, path, reliability)


        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    transfer_time = departure_time - current_time
                    if transfer_time < MIN_TRANSFER_TIME:
                        continue  # Zu wenig Zeit für Umstieg

                if not is_transfer:
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                # Heuristik für A* (vereinfachte Schätzung der verbleibenden Zeit)
                h = heuristic(neighbor, end_name, stops)

                # Füge den nächsten Knoten zur Priority Queue hinzu
                heapq.heappush(pq, (
                    new_current_time,  # Gesamtbewertung: aktuelle Zeit + Heuristik
                    neighbor,
                    path + [(route_id, departure_time, arrival_time)],
                    new_reliability,
                    route_id,
                    h
                ))

    # Gibt das beste Ergebnis zurück, wenn es existiert, andernfalls eine Fehlermeldung
    if best_result:
        return best_result
    else:
        return float("inf"), [], 0.0  # Keine Route gefunden
    
    
def a_star_on_speed(graph, start_name, end_name, start_time_minutes, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=2, stops_df=None):
    pq = [(start_time_minutes, start_name, [], 1.0, None, 0)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie, Heuristik)
    visited = set()
    best_result = None  # Variable, um das beste Ergebnis zu speichern
    count = 0
    while pq:
        current_time, current_stop, path, reliability, last_route, _ = heapq.heappop(pq)

        if (current_stop, current_time, reliability, last_route) in visited:
            continue
        visited.add((current_stop, current_time, reliability, last_route))

        path = path + [(current_stop, current_time)]
        
        # Überprüfe, ob das Ziel erreicht wurde
        if current_stop == end_name:
            print("possibility")
            count += 1
            # Hier speichern wir das erste Ergebnis, wenn das Ziel erreicht wurde
            if best_result is None or current_time < best_result[0]:
                print("possibility is better then before", best_result)
                # Wenn das Ergebnis besser ist, wird es aktualisiert
                best_result = (current_time, path, reliability) # Zähler zurücksetzen
                
        
            # Überprüfung, ob maximale Anzahl an Iterationen ohne Verbesserung erreicht wurde
        if count > 2:
            print("max ieration for here")
            return best_result  # Rückgabe des aktuellen besten Ergebnisses
 
        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    transfer_time = departure_time - current_time
                    if transfer_time < MIN_TRANSFER_TIME:
                        continue  # Zu wenig Zeit für Umstieg

                if not is_transfer:
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                # Heuristik für A* (vereinfachte Schätzung der verbleibenden Zeit)
                h = heuristic(neighbor, end_name, stops)

                # Füge den nächsten Knoten zur Priority Queue hinzu
                heapq.heappush(pq, (
                    new_current_time,  # Gesamtbewertung: aktuelle Zeit + Heuristik
                    neighbor,
                    path + [(route_id, departure_time, arrival_time)],
                    new_reliability,
                    route_id,
                    h
                ))
                
    # Gibt das beste Ergebnis zurück, wenn es existiert, andernfalls eine Fehlermeldung
    if best_result:
        return best_result
    else:
        return float("inf"), [], 0.0  # Keine Route gefunden



In [500]:


def a_star_with_reliability_fixed(graph, start_name, end_name, start_time_minutes, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=4, stops_df=None):
    pq = [(start_time_minutes, start_name, [], 1.0, None, 0)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie, Heuristik)
    visited = set()
    best_result = None  # Variable, um das beste Ergebnis zu speichern
    # Initialisierung für die Überprüfung der Verbesserungen
    count = 0  # Zähler für Iterationen ohne Verbesserung
   
    
    while pq:
        current_time, current_stop, path, reliability, last_route, _ = heapq.heappop(pq)

        if (current_stop, current_time, reliability, last_route) in visited:
            continue
        visited.add((current_stop, current_time, reliability, last_route))

        path = path + [(current_stop, current_time)]

        # Überprüfe, ob das Ziel erreicht wurde
        if current_stop == end_name:
            print("possibility")
            count += 1
            # Hier speichern wir das erste Ergebnis, wenn das Ziel erreicht wurde
            if best_result is None or reliability > best_result[2]:
                print("possibility is better then before", best_result)
                # Wenn das Ergebnis besser ist, wird es aktualisiert
                best_result = (current_time, path, reliability) # Zähler zurücksetzen
                
        
            # Überprüfung, ob maximale Anzahl an Iterationen ohne Verbesserung erreicht wurde
        if count > 4:
            print("max ieration for here")
            return best_result  # Rückgabe des aktuellen besten Ergebnisses


        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    transfer_time = departure_time - current_time
                    if transfer_time < MIN_TRANSFER_TIME or transfer_time > 60:
                        continue  # Zu wenig Zeit für Umstieg

                if not is_transfer:
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                # Heuristik für A* (vereinfachte Schätzung der verbleibenden Zeit)
                h = heuristic(neighbor, end_name, stops)

                # Füge den nächsten Knoten zur Priority Queue hinzu
                heapq.heappush(pq, (
                    new_current_time,  # Gesamtbewertung: aktuelle Zeit + Heuristik
                    neighbor,
                    path + [(route_id, departure_time, arrival_time)],
                    new_reliability,
                    route_id,
                    h
                ))

    # Gibt das beste Ergebnis zurück, wenn es existiert, andernfalls eine Fehlermeldung
    if best_result:
        return best_result
    else:
        return float("inf"), [], 0.0  # Keine Route gefunden


In [292]:


def a_star_backups(graph, path_fixed):

    backup_routes = []
    
    last_route = None
    backup_reliability = 1.0
    
    for i in range(1, len(path_fixed) - 1, 2):  # Alle Umstiegspunkte durchgehen
        transfer_stop, current  = path_fixed[i - 1]
        # Identifiziere alle Umstiegspunkte in der Route
        # for i in range(1, len(path_fixed) - 1, 2):
        
    
        #adjusted_transfer_time = MIN_TRANSFER_TIME + 15
        #current_stop, current_time = path_fixed[i - 1]
        route_id, departure_time, arrival_time = path_fixed[i]
        next_stop, _ = path_fixed[i + 1]
        transfer_time_inbetween = departure_time - current
        
        
        
        if route_id != last_route:
                print(f"lets go from{transfer_stop}")
                missed_trans_rel = 1 - (compute_transfer_probability_with_departure_delay(transfer_time_inbetween))
                backup_time, backup_path, backup_reliability = a_star_with_reliability_fixed(
                    graph, transfer_stop, path_fixed[-1][0], start_time_minutes = departure_time + 1 , stops_df=None, MIN_TRANSFER_TIME= 4,exclude_routes=set()
                )
                backup_routes.append([backup_time, backup_path, backup_reliability])
        last_route = route_id
        
    return backup_routes

Main

In [614]:

# Hauptprogramm
if __name__ == "__main__":

    #agency, stops, routes, trips, stop_times, calendar, calendar_dates, shapes = import_data()
    
    '''
    start_stop_name = "Schattendorf Kirchengasse"
    end_stop_name = "Flughafen Wien Bahnhof"
    start_datetime = "2024-12-12 14:30:00"
    '''
    
    #short
    start_stop_name =  "Wien Hauptbahnhof" 
    end_stop_name =  "Laa/Thaya Bahnhof" 
    start_datetime = "2024-12-12 9:20:00"
    
    '''
    #long 
    start_stop_name = "Kaindorf Bahnhof"
    end_stop_name = "Graz Ostbahnhof"
    start_datetime = "2024-12-12 14:00:00"
    '''

    time_budget = "2:20"
    time_budget_hours, time_budget_minutes = map(int, time_budget.split(":"))
    time_budget_minutes = time_budget_hours * 60 + time_budget_minutes / 60

    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    start_time_minutes = start_time_obj.hour * 60 + start_time_obj.minute

    # In ein reines date-Objekt umwandeln
    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    end_time_obj = start_time_obj + timedelta(minutes=time_budget_minutes)
    date_obj = start_time_obj.date()

    start_time_minutes = start_time_obj.hour * 60 + start_time_obj.minute


'''
    # Ergebnis anzeigen
    if current_time_fast < float("inf"):
        arrival_time_fixed = minutes_to_time(current_time_fast)
        print(f"\n📍 Optimierte zuverlässigste Route von {start_stop_name} nach {end_stop_name}:")

        last_route = None
        grouped_routes = []

        for i in range(1, len(best_result_fast) - 1, 2):
            current_stop, current_time = best_result_fast[i - 1]
            route_id, departure_time, arrival_time = best_result_fast[i]
            next_stop, _ = best_result_fast[i + 1]

            if route_id == last_route:
                grouped_routes[-1]["stops"].append((next_stop, arrival_time))
            else:
                grouped_routes.append({
                    "route_id": route_id,
                    "start_stop": current_stop,
                    "departure_time": departure_time,
                    "stops": [(next_stop, arrival_time)]
                })
            last_route = route_id

        for segment in grouped_routes:
            start = segment["start_stop"]
            dep_time = minutes_to_time(segment["departure_time"])
            route = segment["route_id"]
            stops = " → ".join([f"{stop} (Ankunft: {minutes_to_time(arr)})" for stop, arr in segment["stops"]])
            print(f"  🚆 {start} (Abfahrt: {dep_time}) → {stops} mit Linie {route}")

        print(f"\n🎯 Endstation: {end_stop_name} (Ankunft: {arrival_time_fixed})")
        print(f"🔹 Gesamt-Zuverlässigkeit der Route: {reliability_fast:.2f}\n")

    
    
        
        print("🔄 Backup-Routen:")
        if backup_routes:
            print("\n🔄 Backup-Routen:")
            for stop, path, reliability in backup_routes:
                print(f"  🔁 Backup von {stop}:")
                last_route = None
                first_segment = True

                for segment in path:
                    start, dep_time, end, arr_time, route = segment

                    if last_route == route:
                        print(f" → {end} (Ankunft: {arr_time})", end="")
                    else:
                        if not first_segment:
                            print()
                        print(f"    🚆 {start} (Abfahrt: {dep_time}) → {end} (Ankunft: {arr_time}) mit Linie {route}",
                              end="")
                        first_segment = False

                    last_route = route

                print(f"\n    🔹 Zuverlässigkeit: {reliability:.2f}\n")
        else:
            print("  ❌ Keine Backup-Routen verfügbar.")

        print()
    else:
        print(f"\n⚠️ Keine zuverlässige Route von {start_stop_name} nach {end_stop_name} gefunden.\n")

        '''

'\n    # Ergebnis anzeigen\n    if current_time_fast < float("inf"):\n        arrival_time_fixed = minutes_to_time(current_time_fast)\n        print(f"\n📍 Optimierte zuverlässigste Route von {start_stop_name} nach {end_stop_name}:")\n\n        last_route = None\n        grouped_routes = []\n\n        for i in range(1, len(best_result_fast) - 1, 2):\n            current_stop, current_time = best_result_fast[i - 1]\n            route_id, departure_time, arrival_time = best_result_fast[i]\n            next_stop, _ = best_result_fast[i + 1]\n\n            if route_id == last_route:\n                grouped_routes[-1]["stops"].append((next_stop, arrival_time))\n            else:\n                grouped_routes.append({\n                    "route_id": route_id,\n                    "start_stop": current_stop,\n                    "departure_time": departure_time,\n                    "stops": [(next_stop, arrival_time)]\n                })\n            last_route = route_id\n\n        for s

In [615]:
  #slow and correct
print("creating graph")
graph = create_graph_with_schedule(stop_times, stops, trips, calendar, calendar_dates, start_time_obj, end_time_obj)
#if start_stop_name not in graph or end_stop_name not in graph:
#    print("🚨 Ungültige Start- oder Zielhaltestelle!")
        #sys.exit()

creating graph
1907
Rows after time window and service filter: 17608


In [616]:


def a_star_on_speed(graph, start_name, end_name, start_time_minutes, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=2, stops_df=None):
    pq = [(start_time_minutes,start_time_minutes, start_name, [], 1.0, None)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie, Heuristik)
    visited = set()
    best_result = None  # Variable, um das beste Ergebnis zu speichern
    count = 0
    while pq:
        weight, current_time, current_stop, path, reliability, last_route = heapq.heappop(pq)

        if (current_stop, current_time, reliability, last_route) in visited:
            continue
        visited.add((current_stop, current_time, reliability, last_route))

        path = path + [(current_stop, current_time)]
        
        # Überprüfe, ob das Ziel erreicht wurde
        if current_stop == end_name:
            print("possibility")
            count += 1
            # Hier speichern wir das erste Ergebnis, wenn das Ziel erreicht wurde
            if best_result is None or current_time < best_result[0]:
                print("possibility is better then before", best_result)
                # Wenn das Ergebnis besser ist, wird es aktualisiert
                best_result = (current_time, path, reliability) # Zähler zurücksetzen
                
        
            # Überprüfung, ob maximale Anzahl an Iterationen ohne Verbesserung erreicht wurde
        if count > 2:
            print("max ieration for here")
            return best_result  # Rückgabe des aktuellen besten Ergebnisses
 
        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    transfer_time = departure_time - current_time
                    if transfer_time < MIN_TRANSFER_TIME:
                        continue  # Zu wenig Zeit für Umstieg

                if not is_transfer:
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                # Heuristik für A* (vereinfachte Schätzung der verbleibenden Zeit)
                h = heuristic(neighbor, end_name, stops)

                # Füge den nächsten Knoten zur Priority Queue hinzu
                heapq.heappush(pq, (
                    h, # Gesamtbewertung: aktuelle Zeit + Heuristik
                    new_current_time,  
                    neighbor,
                    path + [(route_id, departure_time, arrival_time)],
                    new_reliability,
                    route_id
                    
                ))
                
    # Gibt das beste Ergebnis zurück, wenn es existiert, andernfalls eine Fehlermeldung
    if best_result:
        return best_result
    else:
        return float("inf"), [], 0.0  # Keine Route gefunden




In [617]:


    # Haupt-Dijkstra-Lauf
    #print("A star")
    #current_time_fast, best_result_fast , reliability_fast = a_star(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=3)

current_time_fast, best_result_fast , reliability_fast = a_star_on_speed(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=2,stops_df=None)




KeyboardInterrupt: 

In [609]:
minutes_to_time(current_time_fast)

'16:24'

In [ ]:
reliability_fast

In [608]:
best_result_fast

[('Schattendorf Kirchengasse', 870),
 ('20-SV5-L-j24-1', 883.0, 888.0),
 ('Loipersbach-Schattendorf Bhf.', 888.0),
 ('3-R93-A-j24-1', 894.0, 898.0),
 ('Marz-Rohrbach Bahnhof', 898.0),
 ('3-R93-A-j24-1', 899.0, 901.0),
 ('Mattersburg Bahnhof', 901.0),
 ('3-R93-A-j24-1', 902.0, 903.0),
 ('Mattersburg Nord Bahnhof', 903.0),
 ('3-R93-A-j24-1', 904.0, 907.0),
 ('Wiesen-Sigleß Bahnhof', 907.0),
 ('3-R93-A-j24-1', 907.0, 911.0),
 ('Bad Sauerbrunn Bahnhof', 911.0),
 ('3-R93-A-j24-1', 913.0, 916.0),
 ('Neudörfl Bahnhof', 916.0),
 ('3-R93-A-j24-1', 917.0, 920.0),
 ('Katzelsdorf Bahnhof', 920.0),
 ('3-R93-A-j24-1', 920.0, 925.0),
 ('Wiener Neustadt Hauptbahnhof', 925.0),
 ('10-A12-j24-1', 927.0, 955.0),
 ('Wien Meidling', 955.0),
 ('10-A12-j24-1', 957.0, 962.0),
 ('Wien Hauptbahnhof', 962.0),
 ('10-A3-j24-1', 969.0, 984.0),
 ('Flughafen Wien Bahnhof', 984.0)]

In [550]:
# Backup-Routen berechnen
print("backups")
backup_routes = a_star_backups(graph,best_result_fast)

backups
lets go fromSchattendorf Kirchengasse
lets go fromLoipersbach-Schattendorf Bhf.
lets go fromWiener Neustadt Hauptbahnhof
possibility
possibility is better then before None
possibility
possibility
possibility
possibility
max ieration for here
lets go fromWien Hauptbahnhof
possibility
possibility is better then before None
possibility


In [551]:
backup_routes

[[inf, [], 0.0],
 [inf, [], 0.0],
 [984.0,
  [('Wiener Neustadt Hauptbahnhof', 928.0),
   ('10-A12-j24-1', 930.0, 955.0),
   ('Wien Meidling', 955.0),
   ('10-A12-j24-1', 957.0, 962.0),
   ('Wien Hauptbahnhof', 962.0),
   ('10-A3-j24-1', 969.0, 984.0),
   ('Flughafen Wien Bahnhof', 984.0)],
  np.float64(0.6767601071186498)],
 [987.0,
  [('Wien Hauptbahnhof', 970.0),
   ('10-A1-j24-1', 972.0, 987.0),
   ('Flughafen Wien Bahnhof', 987.0)],
  1.0]]

In [327]:
reliability_fast

np.float64(0.38494001106330406)

In [328]:
total_reliability =compute_total_reliability(reliability_fast, backup_routes)

In [329]:
total_reliability

np.float64(1.0)

In [65]:
#current_time_fast, best_result_fast , reliability_fast = a_star_on_speed(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=3)

In [ ]:
#current_time_slow, best_result_slow , reliability_slow = a_star(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=3)

In [227]:
transfer_stop='Wiener Neustadt Hauptbahnhof'
end_stop = 'Flughafen Wien Bahnhof'
trip_id='10-A1-j24-1'



In [425]:
a_star_with_reliability_fixed(graph, 'Wiener Neustadt Hauptbahnhof', 'Flughafen Wien Bahnhof', 928, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=4, stops_df=None)

possibility
possibility is better then before None
possibility
possibility
possibility
possibility
possibility is better then before (984.0, [('Wiener Neustadt Hauptbahnhof', 928), ('10-A12-j24-1', 930.0, 955.0), ('Wien Meidling', 955.0), ('10-A12-j24-1', 957.0, 962.0), ('Wien Hauptbahnhof', 962.0), ('10-A3-j24-1', 969.0, 984.0), ('Flughafen Wien Bahnhof', 984.0)], np.float64(0.6767601071186498))
possibility
possibility is better then before (987.0, [('Wiener Neustadt Hauptbahnhof', 928), ('10-A12-j24-1', 930.0, 955.0), ('Wien Meidling', 955.0), ('10-A12-j24-1', 957.0, 962.0), ('Wien Hauptbahnhof', 962.0), ('10-A1-j24-1', 972.0, 987.0), ('Flughafen Wien Bahnhof', 987.0)], np.float64(0.8454126954952396))
max ieration for here


(987.0,
 [('Wiener Neustadt Hauptbahnhof', 928),
  ('10-A12-j24-1', 930.0, 955.0),
  ('Wien Meidling', 955.0),
  ('10-A12-j24-1', 957.0, 962.0),
  ('Wien Hauptbahnhof', 962.0),
  ('10-A12-j24-1', 972.0, 987.0),
  ('Flughafen Wien Bahnhof', 987.0)],
 1.0)

In [162]:
end_time_obj



datetime.datetime(2024, 12, 12, 16, 30, 10)

In [ ]:

def generate_instructions(output):
    """Generiert Anweisungen basierend auf der Liste von Stops und Verbindungen."""
    instructions = []
    start_time = None
    current_line = None

    for i, entry in enumerate(output):
        if len(entry) == 2:  # Start oder Zwischenstopp ohne Verbindung
            stop_name, dep_time = entry
            if start_time is None:
                start_time = dep_time
                instructions.append(f"Starte um {minutes_to_time(start_time)} in {stop_name}.")
            else:
                instructions.append(f"Fahre weiter bis {stop_name}.")
        elif len(entry) == 4:  # Verbindung mit Route
            stop_name, dep_time, arr_time, route = entry
            if current_line != route:
                if current_line is not None:
                    instructions[-1] += f" Steige um in Linie {route} um {minutes_to_time(dep_time)}."
                else:
                    instructions.append(f"Fahre mit Linie {route} um {minutes_to_time(dep_time)}.")
                current_line = route
            instructions.append(f"Halte in {stop_name} um {minutes_to_time(arr_time)}.")

    return "\n".join(instructions)

In [368]:
get_available_service_ids(start_time_obj,calendar,calendar_dates)

TypeError: strptime() argument 1 must be str, not datetime.datetime

In [436]:
available_services = get_available_service_ids(start_time_obj.strftime("%Y-%m-%d"), calendar, calendar_dates)

In [444]:
for i in range(len(available_services)):
    if available_services[i] == "TA+n4":
        print(available_services[i],"available")

TA+n4 available


In [442]:
available_services

['TA',
 'TA+00730',
 'TA+00b00',
 'TA+01720',
 'TA+01m20',
 'TA+01w20',
 'TA+02430',
 'TA+02t20',
 'TA+02x20',
 'TA+04030',
 'TA+05j20',
 'TA+08g20',
 'TA+0a230',
 'TA+0a720',
 'TA+0as10',
 'TA+0b700',
 'TA+0br00',
 'TA+0bv20',
 'TA+0C',
 'TA+0c130',
 'TA+0f130',
 'TA+0f730',
 'TA+0fj10',
 'TA+0g630',
 'TA+0g730',
 'TA+0gm10',
 'TA+0h730',
 'TA+0ha20',
 'TA+0i920',
 'TA+0k620',
 'TA+0k630',
 'TA+0k730',
 'TA+0l130',
 'TA+0l730',
 'TA+0m730',
 'TA+0n630',
 'TA+0n730',
 'TA+0nj20',
 'TA+0o130',
 'TA+0o730',
 'TA+0q130',
 'TA+0q630',
 'TA+0rt00',
 'TA+0rw20',
 'TA+0s730',
 'TA+0t130',
 'TA+0t730',
 'TA+0t900',
 'TA+0un00',
 'TA+0vq20',
 'TA+0w630',
 'TA+0x330',
 'TA+0x720',
 'TA+0x730',
 'TA+0xn20',
 'TA+0y130',
 'TA+0yk20',
 'TA+0z630',
 'TA+10b00',
 'TA+11k20',
 'TA+12230',
 'TA+12u20',
 'TA+13j00',
 'TA+13x20',
 'TA+15v20',
 'TA+15y20',
 'TA+16820',
 'TA+16z20',
 'TA+18g20',
 'TA+19430',
 'TA+19730',
 'TA+1a230',
 'TA+1ac10',
 'TA+1as10',
 'TA+1bp20',
 'TA+1br00',
 'TA+1c130',
 'TA+1c4

In [402]:
len(calendar_dates)

237580

In [403]:
len(calendar)

3351